In [1]:
import pandas as pd
import numpy as np
print("Setup ok")

Setup ok


In [2]:
cargas = pd.DataFrame()
atrac = pd.DataFrame()

for ano in range(2015,2020):    
    cargas_temp = pd.read_csv(f'C:/Users/Sea Group/Documents/TCC/BD_ANTAQ/{ano}/{ano}Carga.txt',sep =";")
    cargas = pd.concat([cargas,cargas_temp], ignore_index=True)
    
    atrac_temp = pd.read_csv(f'C:/Users/Sea Group/Documents/TCC/BD_ANTAQ/{ano}/{ano}Atracacao.txt',sep =";") 
    atrac = pd.concat([atrac,atrac_temp], ignore_index=True)

agri_outlook = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Agricultural Outlook.csv")

comm_price = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Commodity prices, monthly.csv")

pib = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Gross domestic product (GDP).csv")

prod_sales = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Production and Sales (MEI).csv")

taxa_desocupacao = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Taxa de desocupação.csv",sep =";")

infra_invest = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Transport infrastructure investment and maintenance spending.csv")

weel_being_15 = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Well-being 2015.csv")

dolar = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/USD_BRL Dados Históricos.csv")

#Acho que a capacidade de produção de aço não ajuda. Visto que estamos analisando a movimentacao de cntrs
#stell_capacity = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Steel Making Capacity.csv")

print("ok")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12,17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


ok


##########################################  TRATAMENTO DE cargas  #############################################################

In [3]:
#Colunas de cargas

cargas.columns

Index(['IDCarga', 'IDAtracacao', 'Origem', 'Destino', 'CDMercadoria',
       'Tipo Operação da Carga', 'Carga Geral Acondicionamento',
       'ConteinerEstado', 'Tipo Navegação', 'FlagAutorizacao', 'FlagCabotagem',
       'FlagCabotagemMovimentacao', 'FlagConteinerTamanho', 'FlagLongoCurso',
       'FlagMCOperacaoCarga', 'FlagOffshore', 'FlagTransporteViaInterioir',
       'Percurso Transporte em vias Interiores',
       'Percurso Transporte Interiores', 'STNaturezaCarga', 'STSH2', 'STSH4',
       'Natureza da Carga', 'Sentido', 'TEU', 'QTCarga', 'VLPesoCargaBruta'],
      dtype='object')

#Observar que o PIB deve ser linkado com a DATA DE CHEGADA
#Além disso, podemos fazer algumas análises de gráficos pré modelagem para introduzirmos o assunto

#atrac[['Data Atracação', 'Data Chegada',
#       'Data Desatracação', 'Data Início Operação', 'Data Término Operação',
#       'Ano', 'Mes']]

In [4]:
cargas['FlagAutorizacao'].value_counts()

S    9781415
N       1766
Name: FlagAutorizacao, dtype: int64

In [5]:
cargas['FlagConteinerTamanho'].value_counts()

40        4632836
20        2138171
40.0      1118065
20.0       626395
OUTROS     115193
Name: FlagConteinerTamanho, dtype: int64

In [6]:
cargas = cargas[['IDAtracacao', 'Origem', 'Destino',
       'Tipo Operação da Carga', 'Carga Geral Acondicionamento',
       'ConteinerEstado', 'Tipo Navegação', 'FlagConteinerTamanho',
       'Sentido', 'TEU']]

#cargas[['IDCarga', 'IDAtracacao', 'Origem', 'Destino', 'CDMercadoria',
#       'Tipo Operação da Carga', 'Carga Geral Acondicionamento',
#       'ConteinerEstado', 'Tipo Navegação', 'FlagConteinerTamanho',
#       'Sentido', 'TEU', 'QTCarga']].head()

#Colunas não selecionadas:
#       'FlagAutorizacao', 'FlagCabotagem',
#       'FlagCabotagemMovimentacao', 'FlagLongoCurso',
#       'FlagMCOperacaoCarga', 'FlagOffshore', 'FlagTransporteViaInterioir',
#       'Percurso Transporte em vias Interiores'
#Por motido de irrelevância. Em 'Percurso Transporte em vias Interiores' a coluna 'Tipo Navegação' já traz as infos necessárias
#'STNaturezaCarga','STSH2' e 'STSH4': Exclusivo ou Compartilhado. Irrelevante para o projeto
#'Natureza da Carga': totalmente completa como "Carga Conteinerizada". Logo, não é necessária sua presença
#'VLPesoCargaBruta' é proporcional a coluna 'TEU', logo não será utilizada
#'QTCarga' é igual ao TEU
#'IDCarga' foi removida por ser exclusiva

In [7]:
#'Carga Geral Acondicionamento' é dividida em Contenerizada e Solta. 
#As linhas com valores NaN são dropados abaixo
#1667696 (com valor); 46821 (NaN)

cargas = cargas[cargas['Carga Geral Acondicionamento'] == "Conteinerizada"]

#Tira-se os TEUs decimais 
cargas = cargas.loc[(cargas['FlagConteinerTamanho'] != 'OUTROS') &
                    (cargas['FlagConteinerTamanho'].notnull())]
cargas['FlagConteinerTamanho'].replace({'40.0':'40','20.0':'20'}, inplace=True)

#Ajuste do tipo da coluna
cargas['FlagConteinerTamanho'] = cargas['FlagConteinerTamanho'].astype('int')
cargas['TEU'] = cargas['TEU'].astype('int')

#Dropa a coluna 'Carga Geral Acondicionamento', já que terá valor único depois da seleção acima
cargas.drop(columns=['Carga Geral Acondicionamento'], inplace=True)

cargas.reset_index(drop=True, inplace=True)
cargas.head()

,IDAtracacao,Origem,Destino,Tipo Operação da Carga,ConteinerEstado,Tipo Navegação,FlagConteinerTamanho,Sentido,TEU
0,659251,BRSP008,BRCE001,Transbordo,Cheio,Cabotagem,20,Embarcados,29
1,659251,BRSP008,BRCE001,Transbordo,Cheio,Cabotagem,40,Embarcados,152
2,659251,BRSP008,BRSUA,Transbordo,Cheio,Cabotagem,40,Embarcados,62
3,659251,BRSP008,BRAM006,Transbordo,Cheio,Cabotagem,40,Embarcados,10
4,659251,BRSP008,BRAM006,Transbordo,Cheio,Cabotagem,20,Embarcados,15


In [8]:
#As colunas 'Origem' e 'Destino' são boas para introdução dos dados. Para o algorítimo não.
#Logo, são retiradas.

cargas.drop(columns=['Origem','Destino'], inplace=True)
cargas.head()

,IDAtracacao,Tipo Operação da Carga,ConteinerEstado,Tipo Navegação,FlagConteinerTamanho,Sentido,TEU
0,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,29
1,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,152
2,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,62
3,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,10
4,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,15


In [9]:
#'ConteinerEstado' é dividida em Cheio e Vazio.
#As linhas com valores NaN foram dropadas abaixo, tanto que a coluna "TEU" está zerada nas linhas com NaN em ConteinerEstado

cargas = cargas[cargas['ConteinerEstado'].notnull()]
#cargas.head()

In [10]:
#'FlagConteinerTamanho' é complementada por 'TEU' e 'QTCarga'
#A coluna tem o valor "OUTROS" que pode ganhar o valor de 20 ou 40 a depender das colunas complementares
#Quando isso for feito, deve-se mudar o tipo da coluna para float
#Os valores OUTROS equivalem a pouco menos do que 1% dos dados.

cargas['FlagConteinerTamanho'].value_counts()

40    5666508
20    2763130
Name: FlagConteinerTamanho, dtype: int64

In [11]:
#'Sentido' é dividida em Desembarcados, Embarcados e Não informados
#Os valores "Não Informados" são ignorados abaixo, uma vez q representam nem 2% de cada outro valor e nem 1% de todos os valores

cargas = cargas[cargas['Sentido']!= "Não Informado"]

In [12]:
cargas.head()

#'Tipo Navegação': um mesmo cntr pode ser usado na cabotagem e no longo curso? *********************

,IDAtracacao,Tipo Operação da Carga,ConteinerEstado,Tipo Navegação,FlagConteinerTamanho,Sentido,TEU
0,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,29
1,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,152
2,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,62
3,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,10
4,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,15


In [13]:
cargas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8264735 entries, 0 to 8515466
Data columns (total 7 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   IDAtracacao             int64 
 1   Tipo Operação da Carga  object
 2   ConteinerEstado         object
 3   Tipo Navegação          object
 4   FlagConteinerTamanho    int32 
 5   Sentido                 object
 6   TEU                     int32 
dtypes: int32(2), int64(1), object(4)
memory usage: 441.4+ MB


###############################################################################################################################

##########################################  TRATAMENTO DE atrac  ##############################################################

In [14]:
#Colunas de atrac

atrac.columns

#'Nº da Capitania' não tem função para o trabalho

Index(['IDAtracacao', 'CDTUP', 'IDBerco', 'Berço', 'Porto Atracação',
       'Apelido Instalação Portuária', 'Complexo Portuário',
       'Tipo da Autoridade Portuária', 'Data Atracação', 'Data Chegada',
       'Data Desatracação', 'Data Início Operação', 'Data Término Operação',
       'Ano', 'Mes', 'Tipo de Operação', 'Tipo de Navegação da Atracação',
       'Nacionalidade do Armador', 'FlagMCOperacaoAtracacao', 'Terminal',
       'Município', 'UF', 'SGUF', 'Região Geográfica', 'Nº da Capitania',
       'Nº do IMO'],
      dtype='object')

In [15]:
atrac = atrac[['IDAtracacao', 'Porto Atracação',
       'Apelido Instalação Portuária', 'Complexo Portuário',
       'Tipo da Autoridade Portuária', 'Data Atracação', 'Data Chegada',
       'Data Desatracação', 'Data Início Operação', 'Data Término Operação',
       'Ano', 'Mes', 'Tipo de Navegação da Atracação', 'Terminal',
       'Município', 'UF', 'SGUF', 'Região Geográfica']]

#atrac[['IDAtracacao', 'Porto Atracação',
#       'Apelido Instalação Portuária', 'Complexo Portuário',
#       'Tipo da Autoridade Portuária', 'Data Atracação', 'Data Chegada',
#       'Data Desatracação', 'Data Início Operação', 'Data Término Operação',
#       'Ano', 'Mes', 'Tipo de Navegação da Atracação',
#       'Nacionalidade do Armador', 'Terminal',
#       'Município', 'UF', 'SGUF', 'Região Geográfica']].head()

#Como essa tabela é suporte, poucas colunas vão interessar. Uma vez que será extraído dela o local dos portos e terminais.
#Colunas não selecionadas:
#'IDBerco', 'CDTUP', 'Berço', 'Tipo de Operação',
#'FlagMCOperacaoAtracacao', 'Nº da Capitania' e 'Nº do IMO' têm pouca relevância no estudo.

In [16]:
#'Apelido Instalação Portuária' é mantido por ora por segurança.

atrac[atrac['Apelido Instalação Portuária'].isna()]['Complexo Portuário'].value_counts()

Vila do Conde - Belém                   30442
Santos                                  26193
Manaus                                  20846
Rio Grande                              19640
Santarém                                14935
Vitória                                 14435
Porto Velho                             14077
Porto Alegre                            12181
Rio de Janeiro -  Niterói               11678
Paranaguá - Antonina                    11546
Corumbá                                 10078
Itaqui                                   5635
Não Classificado                         5409
São Francisco do Sul                     5199
São João da Barra                        4663
Aratu - Salvador                         3574
Pecém - Fortaleza                        2767
São Simão                                2732
Itajaí                                   2005
Óbidos                                   1983
Itaguaí                                  1820
Maceió                            

In [18]:
#Pega apenas "dd" da Data Inicio Operacao, e renomeia a coluna
atrac['Data Início Operação'] = atrac['Data Início Operação'].str.slice(0,2)
atrac.rename(columns={'Data Início Operação':'Dia'}, inplace=True)

In [19]:
atrac.head()

,IDAtracacao,Porto Atracação,Apelido Instalação Portuária,Complexo Portuário,Tipo da Autoridade Portuária,Data Atracação,Data Chegada,Data Desatracação,Dia,Data Término Operação,Ano,Mes,Tipo de Navegação da Atracação,Terminal,Município,UF,SGUF,Região Geográfica
0,678383,Terminal Graneleiro Hermasa,NaN,Manaus,Porto Privado,08/03/2015 04:32:00,08/03/2015 04:32:00,08/03/2015 06:40:00,08,08/03/2015 06:40:00,2015,mar,Interior,Terminal Graneleiro Hermasa,Itacoatiara,Amazonas,AM,Norte
1,686657,Carinhoso,NaN,Manaus,Porto Privado,20/04/2015 06:00:00,20/04/2015 05:00:00,20/04/2015 07:00:00,20,20/04/2015 07:00:00,2015,abr,Interior,Carinhoso,Manaus,Amazonas,AM,Norte
2,686656,Carinhoso,NaN,Manaus,Porto Privado,30/04/2015 04:00:00,30/04/2015 03:00:00,30/04/2015 05:00:00,30,30/04/2015 05:00:00,2015,abr,Interior,Carinhoso,Manaus,Amazonas,AM,Norte
3,720152,Terminal Graneleiro Hermasa,NaN,Manaus,Porto Privado,05/10/2015 15:00:00,05/10/2015 15:00:00,05/10/2015 17:40:00,05,05/10/2015 17:40:00,2015,out,Interior,Terminal Graneleiro Hermasa,Itacoatiara,Amazonas,AM,Norte
4,703856,Terminal Graneleiro Hermasa,NaN,Manaus,Porto Privado,19/07/2015 04:18:00,19/07/2015 04:18:00,19/07/2015 06:35:00,19,19/07/2015 06:35:00,2015,jul,Interior,Terminal Graneleiro Hermasa,Itacoatiara,Amazonas,AM,Norte


In [20]:
atrac['Tipo de Navegação da Atracação'].value_counts()

Interior           165264
Longo Curso        102140
Cabotagem           51050
Apoio Portuário     25901
Apoio Marítimo      20335
Name: Tipo de Navegação da Atracação, dtype: int64

In [21]:
#Dia, Mes e Ano foram estraídas para se unirem com as demais tabelas. Logo Data de Atracação, Chegada, Desatracação e Término 
#podem ser dropadas
#'Tipo de Autoridade Portuária' é dividida em Público e Privado. Mas mais uma vez é um bom dado para análise. Logo, dropado
#'Tipo de Navegação da Atracação' é divido em 5 tipos (Interior,Longo Curso,Cabotagem,Apoio Portuário,Apoio Marítimo)
#sem muitos efeitos na nossa análise principal.

atrac.drop(columns=['Data Atracação','Data Chegada',
                    'Data Desatracação','Data Término Operação',
                    'Tipo da Autoridade Portuária', 'Tipo de Navegação da Atracação'],inplace=True)

###############################################################################################################################

Juntando cargas e atrac

In [22]:
cargas_atrac = cargas.merge(atrac, how='inner', on='IDAtracacao')

In [23]:
cargas_atrac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8264735 entries, 0 to 8264734
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   IDAtracacao                   int64 
 1   Tipo Operação da Carga        object
 2   ConteinerEstado               object
 3   Tipo Navegação                object
 4   FlagConteinerTamanho          int32 
 5   Sentido                       object
 6   TEU                           int32 
 7   Porto Atracação               object
 8   Apelido Instalação Portuária  object
 9   Complexo Portuário            object
 10  Dia                           object
 11  Ano                           int64 
 12  Mes                           object
 13  Terminal                      object
 14  Município                     object
 15  UF                            object
 16  SGUF                          object
 17  Região Geográfica             object
dtypes: int32(2), int64(2), object(14)
memory u

In [24]:
cargas_atrac['Mes'].value_counts()

ago    733452
out    718062
jul    715694
dez    701740
mai    695371
nov    692906
set    684963
abr    680763
jun    676592
jan    666875
mar    665699
fev    632618
Name: Mes, dtype: int64

In [25]:
cargas_atrac.columns

Index(['IDAtracacao', 'Tipo Operação da Carga', 'ConteinerEstado',
       'Tipo Navegação', 'FlagConteinerTamanho', 'Sentido', 'TEU',
       'Porto Atracação', 'Apelido Instalação Portuária', 'Complexo Portuário',
       'Dia', 'Ano', 'Mes', 'Terminal', 'Município', 'UF', 'SGUF',
       'Região Geográfica'],
      dtype='object')

In [26]:
cargas_atrac.head()

,IDAtracacao,Tipo Operação da Carga,ConteinerEstado,Tipo Navegação,FlagConteinerTamanho,Sentido,TEU,Porto Atracação,Apelido Instalação Portuária,Complexo Portuário,Dia,Ano,Mes,Terminal,Município,UF,SGUF,Região Geográfica
0,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,29,DP World Santos,Ex-Embraport,Santos,03,2015,jan,DP World Santos,Santos,São Paulo,SP,Sudeste
1,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,152,DP World Santos,Ex-Embraport,Santos,03,2015,jan,DP World Santos,Santos,São Paulo,SP,Sudeste
2,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,62,DP World Santos,Ex-Embraport,Santos,03,2015,jan,DP World Santos,Santos,São Paulo,SP,Sudeste
3,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,10,DP World Santos,Ex-Embraport,Santos,03,2015,jan,DP World Santos,Santos,São Paulo,SP,Sudeste
4,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,15,DP World Santos,Ex-Embraport,Santos,03,2015,jan,DP World Santos,Santos,São Paulo,SP,Sudeste


In [42]:
#A base fica muito pesada ao longo do tratamento, como o projeto é estudar apenas as movimentações portuárias em Santos, 
#é sugerido que apenas dados com Município = Santos sejam mantidos. 

cargas_atrac = cargas_atrac.loc[(cargas_atrac['Município'] == 'Santos') |
                                  (cargas_atrac['Município'] =='Guarujá')]

#Como é setado um município específico, as colunas abaixo se tornam obsoletas. Portanto, dropadas. Bem como 'Complexo Portuário'
#A coluna 'Terminal' = 'Porto Atracação' tem apenas DP W.Santos e Santos. Sendo que DP está no Guarujá.
#As colunas também são dropadas
#'Complexo Portuário' só tem 'Santos' como valor, também é dropado

cargas_atrac.drop(columns=['Município','UF',
                           'SGUF','Região Geográfica',
                           'Terminal','Complexo Portuário', 
                           'Porto Atracação'], inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [43]:
cargas_atrac.head()

,IDAtracacao,Tipo Operação da Carga,ConteinerEstado,Tipo Navegação,FlagConteinerTamanho,Sentido,TEU,Apelido Instalação Portuária,Dia,Ano,Mes
0,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,29,Ex-Embraport,03,2015,jan
1,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,152,Ex-Embraport,03,2015,jan
2,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,62,Ex-Embraport,03,2015,jan
3,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,10,Ex-Embraport,03,2015,jan
4,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,15,Ex-Embraport,03,2015,jan


********************************
Eliminar colunas pouco úteis/inuteis para o ML: Origem e Destino (pouco importa de onde e para onde a carga veio e vai);
    Tipo de Op. da Carga e Tipo de Naveg. da Atrac. (não importa pois vislumbramos a movimentação total no porto, não fragmentada);
    Sentido (pouco importante pelo mesmo motivo acima);
    Terminal-Reg.Geo (uma vez que o local de estudo já foi definido)
    
    
Avaliar colunas das bases commodities price e sales. 

Alternativa de solução caso o erro que estamos enfrentando continue 

In [45]:
cargas_atrac.head()

,IDAtracacao,Tipo Operação da Carga,ConteinerEstado,Tipo Navegação,FlagConteinerTamanho,Sentido,TEU,Apelido Instalação Portuária,Dia,Ano,Mes
0,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,29,Ex-Embraport,03,2015,jan
1,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,152,Ex-Embraport,03,2015,jan
2,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,62,Ex-Embraport,03,2015,jan
3,659251,Transbordo,Cheio,Cabotagem,40,Embarcados,10,Ex-Embraport,03,2015,jan
4,659251,Transbordo,Cheio,Cabotagem,20,Embarcados,15,Ex-Embraport,03,2015,jan


###############################################################################################################################

################################################ Agri Outlook ##################################################################

In [46]:
#Colunas do agri outlook

agri_outlook.columns

Index(['COUNTRY', 'Country', 'COMMODITY', 'Commodity', 'VARIABLE', 'Variable',
       'TIME', 'Time', 'Value', 'Flag Codes', 'Flags'],
      dtype='object')

In [47]:
agri_outlook = agri_outlook[['Commodity','Variable','Time','Value']] 

#Mantive 'Variable' pois soa interessante o porque do dado

In [48]:
agri_outlook['Variable'].value_counts()

Production, kt (for Biofuels in millions of litres)                           125
Exports, kt (for Biofuels in millions of litres)                              110
Producer price, local currency/t (for Biofuels local currency/ 100 litres)    110
Imports, kt (for Biofuels in millions of litres)                              110
Consumption, kt (for Biofuels in millions of litres)                          105
Trade balance, kt (for Biofuels in millions of litres)                        100
Food, kt                                                                       80
Consumption per capita, kg                                                     75
Ending stocks, kt (for Biofuels in millions of litres)                         65
Yield                                                                          40
Other use, kt (for Biofuels in millions of litres)                             30
Feed, kt                                                                       25
Area harvested, 

In [49]:
novos_nomes = agri_outlook['Variable'].str.split(",",expand=True)

In [50]:
novos_nomes[0]

0           Production
1           Production
2           Production
3           Production
4           Production
             ...      
1035    Producer price
1036    Producer price
1037    Producer price
1038    Producer price
1039    Producer price
Name: 0, Length: 1040, dtype: object

In [51]:
agri_outlook['Variable']=novos_nomes[0]

In [52]:
agri_outlook['Chave_var_comm'] = agri_outlook['Commodity']+" - "+agri_outlook['Variable']

In [53]:
agri_outlook.drop(columns=['Variable','Commodity'], inplace=True)

In [54]:
df_dummies_agri = pd.get_dummies(agri_outlook,columns=['Chave_var_comm'])

In [55]:
df_dummies_agri.head()

,Time,Value,Chave_var_comm_Beef and veal (cwe) - Consumption,Chave_var_comm_Beef and veal (cwe) - Consumption per capita,Chave_var_comm_Beef and veal (cwe) - Cow inventory,Chave_var_comm_Beef and veal (cwe) - Ending stocks,Chave_var_comm_Beef and veal (cwe) - Exports,Chave_var_comm_Beef and veal (cwe) - Food,Chave_var_comm_Beef and veal (cwe) - Imports,Chave_var_comm_Beef and veal (cwe) - Producer price,...,Chave_var_comm_White sugar (rse) - Producer price,Chave_var_comm_White sugar (rse) - Trade balance,Chave_var_comm_Wholemilk powder (pw) - Consumption,Chave_var_comm_Wholemilk powder (pw) - Consumption per capita,Chave_var_comm_Wholemilk powder (pw) - Exports,Chave_var_comm_Wholemilk powder (pw) - Food,Chave_var_comm_Wholemilk powder (pw) - Imports,Chave_var_comm_Wholemilk powder (pw) - Producer price,Chave_var_comm_Wholemilk powder (pw) - Production,Chave_var_comm_Wholemilk powder (pw) - Trade balance
0,2015,5423.046719,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016,5659.012314,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017,5642.048576,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018,5791.486958,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019,5733.874286,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
for i in range(0,len(df_dummies_agri)):
    for j in range(2,len(df_dummies_agri.columns.to_list())):
        df_dummies_agri.iloc[i,j] = df_dummies_agri.iloc[i,1] * df_dummies_agri.iloc[i,j]  

In [57]:
df_dummies_agri_agg = df_dummies_agri.groupby('Time').sum()
df_dummies_agri_agg.drop(columns=['Value'],inplace=True)

In [58]:
df_dummies_agri_agg.head()

,Chave_var_comm_Beef and veal (cwe) - Consumption,Chave_var_comm_Beef and veal (cwe) - Consumption per capita,Chave_var_comm_Beef and veal (cwe) - Cow inventory,Chave_var_comm_Beef and veal (cwe) - Ending stocks,Chave_var_comm_Beef and veal (cwe) - Exports,Chave_var_comm_Beef and veal (cwe) - Food,Chave_var_comm_Beef and veal (cwe) - Imports,Chave_var_comm_Beef and veal (cwe) - Producer price,Chave_var_comm_Beef and veal (cwe) - Production,Chave_var_comm_Beef and veal (cwe) - Trade balance,...,Chave_var_comm_White sugar (rse) - Producer price,Chave_var_comm_White sugar (rse) - Trade balance,Chave_var_comm_Wholemilk powder (pw) - Consumption,Chave_var_comm_Wholemilk powder (pw) - Consumption per capita,Chave_var_comm_Wholemilk powder (pw) - Exports,Chave_var_comm_Wholemilk powder (pw) - Food,Chave_var_comm_Wholemilk powder (pw) - Imports,Chave_var_comm_Wholemilk powder (pw) - Producer price,Chave_var_comm_Wholemilk powder (pw) - Production,Chave_var_comm_Wholemilk powder (pw) - Trade balance
Time,,,,,,,,,,,,,,,,,,,,,
2015,8976.079060,30.907241,52730.65745,0.0,1725.996754,8976.079060,93.819177,6572.366737,10608.25664,1632.177578,...,1151.206988,9643.371758,593.782421,2.920807,9.616232,593.782421,47.908834,6791.639220,555.489819,-38.292602
2016,9069.934419,30.996547,52371.83229,0.0,1729.482612,9069.934419,94.313580,6619.278597,10705.10345,1635.169032,...,1044.756828,9628.525938,606.658695,2.961799,9.321286,606.658695,48.929020,7068.430159,567.050961,-39.607734
2017,9208.506177,31.244412,52801.46068,0.0,1802.482784,9208.506177,94.492876,6972.223818,10916.49609,1707.989908,...,1087.976629,9083.088569,618.811304,2.999462,9.819252,618.811304,49.440650,7360.301315,579.189906,-39.621398
2018,9320.236587,31.406417,52529.80261,0.0,1835.443246,9320.236587,94.559498,7274.130990,11061.12034,1740.883748,...,1141.558778,10378.108060,630.495061,3.035115,10.378969,630.495061,50.501973,7634.814315,590.372057,-40.123003
2019,9339.610847,31.264778,52332.69831,0.0,1924.315717,9339.610847,94.584882,7406.214783,11169.34168,1829.730835,...,1166.874293,10781.395620,643.328290,3.076530,10.671056,643.328290,51.509701,7935.485765,602.489645,-40.838645


In [59]:
df_dummies_agri_agg.columns.to_list()

['Chave_var_comm_Beef and veal (cwe) - Consumption',
 'Chave_var_comm_Beef and veal (cwe) - Consumption per capita',
 'Chave_var_comm_Beef and veal (cwe) - Cow inventory',
 'Chave_var_comm_Beef and veal (cwe) - Ending stocks',
 'Chave_var_comm_Beef and veal (cwe) - Exports',
 'Chave_var_comm_Beef and veal (cwe) - Food',
 'Chave_var_comm_Beef and veal (cwe) - Imports',
 'Chave_var_comm_Beef and veal (cwe) - Producer price',
 'Chave_var_comm_Beef and veal (cwe) - Production',
 'Chave_var_comm_Beef and veal (cwe) - Trade balance',
 'Chave_var_comm_Biodiesel - Consumption',
 'Chave_var_comm_Biodiesel - Producer price',
 'Chave_var_comm_Biodiesel - Production',
 'Chave_var_comm_Biodiesel - Trade balance',
 'Chave_var_comm_Butter (pw) - Consumption',
 'Chave_var_comm_Butter (pw) - Consumption per capita',
 'Chave_var_comm_Butter (pw) - Exports',
 'Chave_var_comm_Butter (pw) - Food',
 'Chave_var_comm_Butter (pw) - Imports',
 'Chave_var_comm_Butter (pw) - Producer price',
 'Chave_var_comm_Butt

In [119]:
#Apenas algumas colunas são mantidas, elas são as ligadas à importação proteica do BR. As demais são transportadas via granel
# Mantivemos a os peixes pois: http://www.paranacooperativo.coop.br/ppc/index.php/sistema-ocepar/comunicacao/2011-12-07-11-06-29/ultimas-noticias/137287-piscicultura-exportacoes-de-peixes-ja-sao-10-maiores-que-em-2020

df_dummies_agri_agg.drop(columns=['Chave_var_comm_Biodiesel - Consumption',
                                 'Chave_var_comm_Biodiesel - Producer price',
                                 'Chave_var_comm_Biodiesel - Production',
                                 'Chave_var_comm_Biodiesel - Trade balance',
                                 'Chave_var_comm_Butter (pw) - Consumption',
                                 'Chave_var_comm_Butter (pw) - Consumption per capita',
                                 'Chave_var_comm_Butter (pw) - Exports',
                                 'Chave_var_comm_Butter (pw) - Food',
                                 'Chave_var_comm_Butter (pw) - Imports',
                                 'Chave_var_comm_Butter (pw) - Producer price',
                                 'Chave_var_comm_Butter (pw) - Production',
                                 'Chave_var_comm_Butter (pw) - Trade balance',
                                 'Chave_var_comm_Cheese (pw) - Consumption',
                                 'Chave_var_comm_Cheese (pw) - Consumption per capita',
                                 'Chave_var_comm_Cheese (pw) - Exports',
                                 'Chave_var_comm_Cheese (pw) - Food',
                                 'Chave_var_comm_Cheese (pw) - Imports',
                                 'Chave_var_comm_Cheese (pw) - Producer price',
                                 'Chave_var_comm_Cheese (pw) - Production',
                                 'Chave_var_comm_Cheese (pw) - Trade balance',
                                 'Chave_var_comm_Coarse grains - Area harvested',
                                 'Chave_var_comm_Coarse grains - Biofuel use',
                                 'Chave_var_comm_Coarse grains - Consumption',
                                 'Chave_var_comm_Coarse grains - Consumption per capita',
                                 'Chave_var_comm_Coarse grains - Ending stocks',
                                 'Chave_var_comm_Coarse grains - Exports',
                                 'Chave_var_comm_Coarse grains - Feed',
                                 'Chave_var_comm_Coarse grains - Food',
                                 'Chave_var_comm_Coarse grains - Imports',
                                 'Chave_var_comm_Coarse grains - Other use',
                                 'Chave_var_comm_Coarse grains - Producer price',
                                 'Chave_var_comm_Coarse grains - Production',
                                 'Chave_var_comm_Coarse grains - Trade balance',
                                 'Chave_var_comm_Coarse grains - Yield',
                                 'Chave_var_comm_Ethanol - Consumption',
                                 'Chave_var_comm_Ethanol - Other use',
                                 'Chave_var_comm_Ethanol - Producer price',
                                 'Chave_var_comm_Ethanol - Production',
                                 'Chave_var_comm_Ethanol - Trade balance',
                                 'Chave_var_comm_Fish oil - Consumption',
                                 'Chave_var_comm_Fish oil - Ending stocks',
                                 'Chave_var_comm_Fish oil - Exports',
                                 'Chave_var_comm_Fish oil - Imports',
                                 'Chave_var_comm_Fish oil - Producer price',
                                 'Chave_var_comm_Fish oil - Production',
                                 'Chave_var_comm_Fish oil - Yield',
                                 'Chave_var_comm_Fresh dairy products - Consumption',
                                 'Chave_var_comm_Fresh dairy products - Consumption per capita',
                                 'Chave_var_comm_Fresh dairy products - Exports',
                                 'Chave_var_comm_Fresh dairy products - Food',
                                 'Chave_var_comm_Fresh dairy products - Imports',
                                 'Chave_var_comm_Fresh dairy products - Production',
                                 'Chave_var_comm_Milk - Cow inventory',
                                 'Chave_var_comm_Milk - Producer price',
                                 'Chave_var_comm_Milk - Production',
                                 'Chave_var_comm_Milk - Yield',
                                 'Chave_var_comm_Oilseeds - Area harvested',
                                 'Chave_var_comm_Oilseeds - Consumption',
                                 'Chave_var_comm_Oilseeds - Crush',
                                 'Chave_var_comm_Oilseeds - Ending stocks',
                                 'Chave_var_comm_Oilseeds - Exports',
                                 'Chave_var_comm_Oilseeds - Food',
                                 'Chave_var_comm_Oilseeds - Imports',
                                 'Chave_var_comm_Oilseeds - Other use',
                                 'Chave_var_comm_Oilseeds - Producer price',
                                 'Chave_var_comm_Oilseeds - Production',
                                 'Chave_var_comm_Oilseeds - Trade balance',
                                 'Chave_var_comm_Oilseeds - Yield','Chave_var_comm_Raw sugar (rse) - Exports',
                                 'Chave_var_comm_Raw sugar (rse) - Imports',
                                 'Chave_var_comm_Raw sugar (rse) - Producer price',
                                 'Chave_var_comm_Raw sugar (rse) - Trade balance',
                                 'Chave_var_comm_Rice - Area harvested',
                                 'Chave_var_comm_Rice - Consumption',
                                 'Chave_var_comm_Rice - Consumption per capita',
                                 'Chave_var_comm_Rice - Ending stocks',
                                 'Chave_var_comm_Rice - Exports',
                                 'Chave_var_comm_Rice - Feed',
                                 'Chave_var_comm_Rice - Food',
                                 'Chave_var_comm_Rice - Imports',
                                 'Chave_var_comm_Rice - Producer price',
                                 'Chave_var_comm_Rice - Production',
                                 'Chave_var_comm_Rice - Trade balance',
                                 'Chave_var_comm_Rice - Yield',
                                 'Chave_var_comm_Sheepmeat(cwe) - Consumption',
                                 'Chave_var_comm_Sheepmeat(cwe) - Consumption per capita',
                                 'Chave_var_comm_Sheepmeat(cwe) - Ending stocks',
                                 'Chave_var_comm_Sheepmeat(cwe) - Exports',
                                 'Chave_var_comm_Sheepmeat(cwe) - Food',
                                 'Chave_var_comm_Sheepmeat(cwe) - Imports',
                                 'Chave_var_comm_Sheepmeat(cwe) - Producer price',
                                 'Chave_var_comm_Sheepmeat(cwe) - Production',
                                 'Chave_var_comm_Sheepmeat(cwe) - Trade balance',
                                 'Chave_var_comm_Skim milk powder (pw) - Consumption',
                                 'Chave_var_comm_Skim milk powder (pw) - Consumption per capita',
                                 'Chave_var_comm_Skim milk powder (pw) - Exports',
                                 'Chave_var_comm_Skim milk powder (pw) - Feed',
                                 'Chave_var_comm_Skim milk powder (pw) - Food',
                                 'Chave_var_comm_Skim milk powder (pw) - Imports',
                                 'Chave_var_comm_Skim milk powder (pw) - Producer price',
                                 'Chave_var_comm_Skim milk powder (pw) - Production',
                                 'Chave_var_comm_Skim milk powder (pw) - Trade balance',
                                 'Chave_var_comm_Sugar (rse) - Consumption',
                                 'Chave_var_comm_Sugar (rse) - Consumption per capita',
                                 'Chave_var_comm_Sugar (rse) - Ending stocks',
                                 'Chave_var_comm_Sugar (rse) - Exports',
                                 'Chave_var_comm_Sugar (rse) - Food',
                                 'Chave_var_comm_Sugar (rse) - Imports',
                                 'Chave_var_comm_Sugar (rse) - Production',
                                 'Chave_var_comm_Sugar (rse) - Trade balance',
                                 'Chave_var_comm_Sugar cane - Area harvested',
                                 'Chave_var_comm_Sugar cane - Biofuel use',
                                 'Chave_var_comm_Sugar cane - Producer price',
                                 'Chave_var_comm_Sugar cane - Production',
                                 'Chave_var_comm_Sugar cane - Yield',
                                 'Chave_var_comm_Vegetable oils - Biofuel use',
                                 'Chave_var_comm_Vegetable oils - Consumption',
                                 'Chave_var_comm_Vegetable oils - Consumption per capita',
                                 'Chave_var_comm_Vegetable oils - Ending stocks',
                                 'Chave_var_comm_Vegetable oils - Exports',
                                 'Chave_var_comm_Vegetable oils - Food',
                                 'Chave_var_comm_Vegetable oils - Imports',
                                 'Chave_var_comm_Vegetable oils - Other use',
                                 'Chave_var_comm_Vegetable oils - Producer price',
                                 'Chave_var_comm_Vegetable oils - Production',
                                 'Chave_var_comm_Vegetable oils - Trade balance',
                                 'Chave_var_comm_Wheat - Area harvested',
                                 'Chave_var_comm_Wheat - Biofuel use',
                                 'Chave_var_comm_Wheat - Consumption',
                                 'Chave_var_comm_Wheat - Consumption per capita',
                                 'Chave_var_comm_Wheat - Ending stocks',
                                 'Chave_var_comm_Wheat - Exports',
                                 'Chave_var_comm_Wheat - Feed',
                                 'Chave_var_comm_Wheat - Food',
                                 'Chave_var_comm_Wheat - Imports',
                                 'Chave_var_comm_Wheat - Other use',
                                 'Chave_var_comm_Wheat - Producer price',
                                 'Chave_var_comm_Wheat - Production',
                                 'Chave_var_comm_Wheat - Trade balance',
                                 'Chave_var_comm_Wheat - Yield',
                                 'Chave_var_comm_Whey powder (pw) - Exports',
                                 'Chave_var_comm_Whey powder (pw) - Imports',
                                 'Chave_var_comm_Whey powder (pw) - Trade balance',
                                 'Chave_var_comm_White sugar (rse) - Exports',
                                 'Chave_var_comm_White sugar (rse) - Imports',
                                 'Chave_var_comm_White sugar (rse) - Producer price',
                                 'Chave_var_comm_White sugar (rse) - Trade balance',
                                 'Chave_var_comm_Wholemilk powder (pw) - Consumption',
                                 'Chave_var_comm_Wholemilk powder (pw) - Consumption per capita',
                                 'Chave_var_comm_Wholemilk powder (pw) - Exports',
                                 'Chave_var_comm_Wholemilk powder (pw) - Food',
                                 'Chave_var_comm_Wholemilk powder (pw) - Imports',
                                 'Chave_var_comm_Wholemilk powder (pw) - Producer price',
                                 'Chave_var_comm_Wholemilk powder (pw) - Production',
                                 'Chave_var_comm_Wholemilk powder (pw) - Trade balance',
                                
                                    ],inplace=True)

KeyError: "['Chave_var_comm_Biodiesel - Consumption'\n 'Chave_var_comm_Biodiesel - Producer price'\n 'Chave_var_comm_Biodiesel - Production'\n 'Chave_var_comm_Biodiesel - Trade balance'\n 'Chave_var_comm_Butter (pw) - Consumption'\n 'Chave_var_comm_Butter (pw) - Consumption per capita'\n 'Chave_var_comm_Butter (pw) - Exports'\n 'Chave_var_comm_Butter (pw) - Food'\n 'Chave_var_comm_Butter (pw) - Imports'\n 'Chave_var_comm_Butter (pw) - Producer price'\n 'Chave_var_comm_Butter (pw) - Production'\n 'Chave_var_comm_Butter (pw) - Trade balance'\n 'Chave_var_comm_Cheese (pw) - Consumption'\n 'Chave_var_comm_Cheese (pw) - Consumption per capita'\n 'Chave_var_comm_Cheese (pw) - Exports'\n 'Chave_var_comm_Cheese (pw) - Food'\n 'Chave_var_comm_Cheese (pw) - Imports'\n 'Chave_var_comm_Cheese (pw) - Producer price'\n 'Chave_var_comm_Cheese (pw) - Production'\n 'Chave_var_comm_Cheese (pw) - Trade balance'\n 'Chave_var_comm_Coarse grains - Area harvested'\n 'Chave_var_comm_Coarse grains - Biofuel us

In [ ]:
df_dummies_agri_agg.drop(columns=['Chave_var_comm_Beef and veal (cwe) - Consumption',
                                'Chave_var_comm_Beef and veal (cwe) - Consumption per capita',
                                'Chave_var_comm_Beef and veal (cwe) - Cow inventory',
                                'Chave_var_comm_Beef and veal (cwe) - Ending stocks',
                                'Chave_var_comm_Beef and veal (cwe) - Food',
                                'Chave_var_comm_Beef and veal (cwe) - Producer price',
                                'Chave_var_comm_Beef and veal (cwe) - Production',
                                'Chave_var_comm_Beef and veal (cwe) - Trade balance',
                                'Chave_var_comm_Fish - Consumption',
                                'Chave_var_comm_Fish - Consumption per capita',
                                'Chave_var_comm_Fish - Crush',
                                'Chave_var_comm_Fish - Food',
                                'Chave_var_comm_Fish - Other use',
                                'Chave_var_comm_Fish - Producer price',
                                'Chave_var_comm_Fish - Production',
                                'Chave_var_comm_Fish from aquaculture - Producer price',
                                'Chave_var_comm_Fish from aquaculture - Production',
                                'Chave_var_comm_Fish from capture - Production',
                                'Chave_var_comm_Fish meal - Consumption',
                                'Chave_var_comm_Fish meal - Ending stocks',
                                'Chave_var_comm_Fish meal - Producer price',
                                'Chave_var_comm_Fish meal - Production',
                                'Chave_var_comm_Fish meal - Yield',
                                'Chave_var_comm_Pigmeat (cwe) - Consumption',
                                'Chave_var_comm_Pigmeat (cwe) - Consumption per capita',
                                'Chave_var_comm_Pigmeat (cwe) - Ending stocks',
                                'Chave_var_comm_Pigmeat (cwe) - Food',
                                'Chave_var_comm_Pigmeat (cwe) - Producer price',
                                'Chave_var_comm_Pigmeat (cwe) - Production',
                                'Chave_var_comm_Pigmeat (cwe) - Trade balance',
                                'Chave_var_comm_Poultry meat (rtc) - Consumption',
                                'Chave_var_comm_Poultry meat (rtc) - Consumption per capita',
                                'Chave_var_comm_Poultry meat (rtc) - Ending stocks',
                                'Chave_var_comm_Poultry meat (rtc) - Food',
                                'Chave_var_comm_Poultry meat (rtc) - Production',
                                'Chave_var_comm_Poultry meat (rtc) - Trade balance',
                                'Chave_var_comm_Protein meals - Consumption',
                                'Chave_var_comm_Protein meals - Ending stocks',
                                'Chave_var_comm_Protein meals - Feed',
                                'Chave_var_comm_Protein meals - Production',
                                'Chave_var_comm_Protein meals - Trade balance'])

In [120]:
df_dummies_agri_agg.columns.to_list()

['Chave_var_comm_Beef and veal (cwe) - Consumption',
 'Chave_var_comm_Beef and veal (cwe) - Consumption per capita',
 'Chave_var_comm_Beef and veal (cwe) - Cow inventory',
 'Chave_var_comm_Beef and veal (cwe) - Ending stocks',
 'Chave_var_comm_Beef and veal (cwe) - Exports',
 'Chave_var_comm_Beef and veal (cwe) - Food',
 'Chave_var_comm_Beef and veal (cwe) - Imports',
 'Chave_var_comm_Beef and veal (cwe) - Producer price',
 'Chave_var_comm_Beef and veal (cwe) - Production',
 'Chave_var_comm_Beef and veal (cwe) - Trade balance',
 'Chave_var_comm_Fish - Consumption',
 'Chave_var_comm_Fish - Consumption per capita',
 'Chave_var_comm_Fish - Crush',
 'Chave_var_comm_Fish - Exports',
 'Chave_var_comm_Fish - Food',
 'Chave_var_comm_Fish - Imports',
 'Chave_var_comm_Fish - Other use',
 'Chave_var_comm_Fish - Producer price',
 'Chave_var_comm_Fish - Production',
 'Chave_var_comm_Fish from aquaculture - Producer price',
 'Chave_var_comm_Fish from aquaculture - Production',
 'Chave_var_comm_Fish 

df_dummies_agri.loc[(df_dummies_agri['Commodity_Biodiesel'] == 1) &
                   (df_dummies_agri['Time'] == 2019)]

In [62]:
cargas_atrac.shape

(2962843, 11)

In [64]:
#Junção da tabela cargas_atrac com a agri
cargas_atrac_agri = cargas_atrac.merge(df_dummies_agri_agg, how='inner', left_on='Ano', right_index=True)

In [ ]:
#cargas_atrac_agri.head()

###############################################################################################################################

################################################ Commod. Price ################################################################

In [ ]:
#Todas as céluas abaixo comentadas pois são pouco uteis já que os commodities são transportadas via granel 

comm_price.columns

comm_price

#Transforma a tabela pivotada numa tabela normal

comm_price = pd.DataFrame(comm_price.to_records())
#comm_price

#Ajuda a tabela para um modo utilizável
#Temos que alterar melhor essa tabela para ela conversar com as demais *******************
#Trocar as linhas pelas colunas

comm_price.columns = comm_price.iloc[0]
comm_price = comm_price.drop([1,2,3,4])
comm_price = comm_price.set_index('PERIOD', drop=True)

#comm_price

comm_price = comm_price.transpose()
comm_price = comm_price.drop('PERIOD', axis=1).reset_index()

#comm_price.head()

#É preciso trocar a coluna PERIOD

meses_comm_price = pd.read_csv("C:/Users/Sea Group/Documents/TCC/OECD/Meses_2015_2019.csv")
#meses_comm_price

#Ajusta o nome da coluna bem como seus valores

comm_price[0] = meses_comm_price['Mes/ano']
comm_price.rename(columns={0:'Mes/ano'}, inplace=True)

comm_price.columns

comm_price.head()

#Coluna Mes/ano tratada
mes_ano_trat = comm_price['Mes/ano'].str.split("/",expand=True)
mes_ano_trat[1] = mes_ano_trat[1].map({'15':2015,'16':2016,'17':2017,'18':2018,'19':2019})
mes_ano_trat

comm_price.drop(columns=['Mes/ano'], inplace=True)
comm_price[['Mes','Ano']] = mes_ano_trat

comm_price.head()

cargas_atrac_agri_cprice = cargas_atrac_agri.merge(comm_price, how='inner', on=['Mes','Ano'])

###############################################################################################################################

################################################ PIB ##########################################################################

In [65]:
pib = pib[['Transaction', 'MEASURE', 'Measure',
       'TIME', 'Year', 'Unit Code', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Unit','Flag Codes',
       'Flags']]

In [66]:
pib['Transaction'].value_counts()

Gross domestic product (expenditure approach)                                  84
Other service activities (ISIC rev4)                                           80
Agriculture, forestry and fishing (ISIC rev4)                                  80
Construction (ISIC rev4)                                                       80
Prof., scientific, techn.; admin., support serv. activities (ISIC rev4)        80
Information and communication (ISIC rev4)                                      80
Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)      80
Public admin.; compulsory s.s.; education; human health (ISIC rev4)            80
Final consumption expenditure of households                                    79
Financial and insurance activities (ISIC rev4)                                 68
Industry, including energy (ISIC rev4)                                         68
Real estate activities (ISIC rev4)                                             68
of which: Manufa

In [67]:
pib.columns

Index(['Transaction', 'MEASURE', 'Measure', 'TIME', 'Year', 'Unit Code',
       'PowerCode Code', 'PowerCode', 'Reference Period Code',
       'Reference Period', 'Value', 'Unit', 'Flag Codes', 'Flags'],
      dtype='object')

In [68]:
pib.head()

,Transaction,MEASURE,Measure,TIME,Year,Unit Code,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Unit,Flag Codes,Flags
0,Domestic demand,C,Current prices,2015,2015,BRL,6,Millions,NaN,NaN,6064933.0,Brazilian Real,NaN,NaN
1,Domestic demand,C,Current prices,2016,2016,BRL,6,Millions,NaN,NaN,6244271.0,Brazilian Real,NaN,NaN
2,Domestic demand,C,Current prices,2017,2017,BRL,6,Millions,NaN,NaN,6538182.0,Brazilian Real,NaN,NaN
3,Domestic demand,C,Current prices,2018,2018,BRL,6,Millions,NaN,NaN,6976559.0,Brazilian Real,NaN,NaN
4,Gross fixed capital formation,C,Current prices,2015,2015,BRL,6,Millions,NaN,NaN,1069397.0,Brazilian Real,NaN,NaN


In [69]:
a = pib['Transaction']

In [70]:
#Chave que contempla a transaction com a unidade de medida
pib['Chave'] = pib['Transaction'] + pib['Unit']

In [71]:
#Ajusta PIB
df_demmies_transaction_name = pd.get_dummies(pib,columns=['Chave'])
col = [0,1,2,3,5,6,7,8,9,11,12,13]
df_demmies_transaction_name.drop(df_demmies_transaction_name.columns[col], axis=1,inplace=True)
df_demmies_transaction_name.head()

,Year,Value,"Chave_Agriculture, forestry and fishing (ISIC rev4)Brazilian Real","Chave_Agriculture, forestry and fishing (ISIC rev4)Index","Chave_Agriculture, forestry and fishing (ISIC rev4)Percentage","Chave_Agriculture, forestry and fishing (ISIC rev4)US Dollar",Chave_Changes in inventories and acquisitions less disposals of valuablesBrazilian Real,Chave_Changes in inventories and acquisitions less disposals of valuablesUS Dollar,Chave_Changes in inventoriesBrazilian Real,Chave_Changes in inventoriesUS Dollar,...,Chave_Wages and salariesBrazilian Real,Chave_Wages and salariesUS Dollar,Chave_of which: Actual individual consumptionBrazilian Real,Chave_of which: Actual individual consumptionUS Dollar,Chave_of which: Manufacturing (ISIC rev 4)Brazilian Real,Chave_of which: Manufacturing (ISIC rev 4)US Dollar,Chave_of which: Manufacturing (ISIC rev4)Brazilian Real,Chave_of which: Manufacturing (ISIC rev4)Index,Chave_of which: Manufacturing (ISIC rev4)Percentage,Chave_of which: Manufacturing (ISIC rev4)US Dollar
0,2015,6064933.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016,6244271.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017,6538182.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018,6976559.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015,1069397.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
#Seleciona as colunas a serem dropadas
drop_list = [col for col in df_demmies_transaction_name.columns if ('Brazilian Real' in col) or
                                                                    ('Percentage' in col) or 
                                                                     ('Index' in col)]

In [73]:
#Drop das especificadas
df_demmies_transaction_name = df_demmies_transaction_name.drop(columns=drop_list)

In [74]:
df_demmies_transaction_name.columns

Index(['Year', 'Value',
       'Chave_Agriculture, forestry and fishing (ISIC rev4)US Dollar',
       'Chave_Changes in inventories and acquisitions less disposals of valuablesUS Dollar',
       'Chave_Changes in inventoriesUS Dollar',
       'Chave_Collective consumption expenditure of general governmentUS Dollar',
       'Chave_Compensation of employeesUS Dollar',
       'Chave_Construction (ISIC rev4)US Dollar',
       'Chave_Cultivated biological resourcesUS Dollar',
       'Chave_Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)US Dollar',
       'Chave_Domestic demandUS Dollar', 'Chave_DwellingsUS Dollar',
       'Chave_Employers' social contributionsUS Dollar',
       'Chave_Exports of goods and servicesUS Dollar',
       'Chave_External balance of goods and servicesUS Dollar',
       'Chave_Final consumption expenditure of general governmentUS Dollar',
       'Chave_Final consumption expenditure of householdsOECD=100',
       'Chave_Final consumption exp

df_demmies_transaction_name.info()

In [75]:
df_demmies_transaction_name['Value'] = df_demmies_transaction_name['Value'].astype(float)
for i in range(0,len(df_demmies_transaction_name)):
    for j in range(2,len(df_demmies_transaction_name.columns.to_list())):
        df_demmies_transaction_name.iloc[i,j] = df_demmies_transaction_name.iloc[i,1] * df_demmies_transaction_name.iloc[i,j]

In [76]:
df_demmies_transaction_name = df_demmies_transaction_name.groupby(by='Year').sum().drop(columns='Value')#, inplace=True)

In [77]:
#Join do pib com as demais
cargas_atrac_agri_pib = cargas_atrac_agri.merge(df_demmies_transaction_name, how='inner',
                                                              left_on='Ano', right_on='Year')

pib.loc[pib['Transaction'] == 'Wages and salaries']

pib['Transaction'].value_counts()

In [ ]:
#Talvez essa base mais atrapalhe do que ajude. Primeiro que são levantamentos anuais,
#segundo que há pouco dados para cada "Transaction" 

In [ ]:
#Os componentes do PIB apresentados nessa tabela talvez não nos ajudem se vistos isoladamente. 
#O valor "condensado" do PIB é mais útil ***************** PROCURAR ESSA INFO *************

#Verificar como um valor singular pode influenciar no algoritimo ML. Como um valor representativo, como o PIB (somente), pode 
#influenciar o ML 
#************************* VER COM O PROFESSOR

pib

###############################################################################################################################

############################################### Prod. Sales ###################################################################

In [78]:
prod_sales.columns

Index(['SUBJECT', 'Subject', 'LOCATION', 'Country', 'FREQUENCY', 'Frequency',
       'TIME', 'Time', 'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')

In [79]:
prod_sales.head()

,SUBJECT,Subject,LOCATION,Country,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,SLRTCR03,"Passenger car registrations sa, Index",AUS,Australia,M,Monthly,2015-01,Jan-2015,IDX,Index,0,Units,2010_100,2010=100,102.37680,NaN,NaN
1,SLRTCR03,"Passenger car registrations sa, Index",AUS,Australia,M,Monthly,2015-02,Feb-2015,IDX,Index,0,Units,2010_100,2010=100,102.19530,NaN,NaN
2,SLRTCR03,"Passenger car registrations sa, Index",AUS,Australia,M,Monthly,2015-03,Mar-2015,IDX,Index,0,Units,2010_100,2010=100,106.07450,NaN,NaN
3,SLRTCR03,"Passenger car registrations sa, Index",AUS,Australia,M,Monthly,2015-04,Apr-2015,IDX,Index,0,Units,2010_100,2010=100,94.42778,NaN,NaN
4,SLRTCR03,"Passenger car registrations sa, Index",AUS,Australia,M,Monthly,2015-05,May-2015,IDX,Index,0,Units,2010_100,2010=100,98.55822,NaN,NaN


In [80]:
#Escolhido a mesma referencia para os dados. Tinhamos outra referencia (2015)
prod_sales =prod_sales.loc[prod_sales['Reference Period']=='2010=100']

In [81]:
prod_sales.loc[prod_sales['Reference Period']=='2010=100']['TIME'].value_counts().sort_values(0)

2019-02     81
2019-09     81
2019-04     81
2019-05     81
2019-03     81
2019-07     81
2019-12     81
2019-08     81
2019-06     81
2019-11     81
2019-10     81
2019-01     81
2018-12    166
2018-10    166
2018-11    166
2018-09    170
2018-08    171
2018-07    172
2018-06    173
2018-05    173
2018-04    173
2018-02    173
2018-03    173
2018-01    173
2017-09    177
2017-11    177
2017-10    177
2017-12    177
2017-08    177
2016-12    178
2016-07    178
2016-08    178
2017-06    178
2017-02    178
2017-04    178
2017-05    178
2017-07    178
2016-06    178
2016-10    178
2017-01    178
2017-03    178
2016-09    178
2016-11    178
2016-02    179
2016-03    179
2016-01    179
2016-04    179
2016-05    179
2015-12    180
2015-03    180
2015-08    180
2015-01    180
2015-11    180
2015-05    180
2015-06    180
2015-09    180
2015-02    180
2015-04    180
2015-10    180
2015-07    180
Name: TIME, dtype: int64

In [82]:
#Dados MENSAIS. Selção das colunas relevantes
#'Reference Period' = '2010=100'

prod_sales = prod_sales[['Subject', 'LOCATION', 'Country',
        'Time', 'Value']]

In [83]:
#Seleção apenas de EUA e UE por serem grandes polos ligados ao COMEX BR

prod_sales = prod_sales.loc[prod_sales['Country'].isin(['Euro area','Brazil', 'United States'])]

In [84]:
#Cria chave para ajustar a tabela
prod_sales['Chave'] = prod_sales['Subject'] + prod_sales['Country']

In [85]:
#Ajusta a tabela
dummies_prod_sales = pd.get_dummies(prod_sales,columns=['Chave'])
dummies_prod_sales.head()

,Subject,LOCATION,Country,Time,Value,"Chave_Passenger car registrations sa, IndexUnited States","Chave_Production of total manufactured intermediate goods sa, IndexUnited States","Chave_Total retail trade (Volume) sa, IndexBrazil","Chave_Total retail trade (Volume) sa, IndexUnited States","Chave_Work started for dwellings sa, IndexUnited States"
9694,Production of total manufactured intermediate ...,USA,United States,Jan-2015,100.79920,0,1,0,0,0
9695,Production of total manufactured intermediate ...,USA,United States,Feb-2015,100.22830,0,1,0,0,0
9696,Production of total manufactured intermediate ...,USA,United States,Mar-2015,100.05110,0,1,0,0,0
9697,Production of total manufactured intermediate ...,USA,United States,Apr-2015,100.12100,0,1,0,0,0
9698,Production of total manufactured intermediate ...,USA,United States,May-2015,99.68279,0,1,0,0,0


In [86]:
dummies_prod_sales['Value'] = dummies_prod_sales['Value'].astype(float)
for i in range(0,len(dummies_prod_sales)):
    for j in range(4,len(dummies_prod_sales.columns.to_list())):
        dummies_prod_sales.iloc[i,j] = dummies_prod_sales.iloc[i,4] * dummies_prod_sales.iloc[i,j]

In [87]:
#Dropa colunas nao mais necessárias
dummies_prod_sales.drop(dummies_prod_sales.columns[[0,1,2,4]], axis=1,inplace=True)

In [88]:
#Alguns dados estao zerados, sugestão: substituir pela média, ou, úlitmo valor
dummies_prod_sales = dummies_prod_sales.groupby(by='Time').sum().reset_index()
dummies_prod_sales.head()

,Time,"Chave_Passenger car registrations sa, IndexUnited States","Chave_Production of total manufactured intermediate goods sa, IndexUnited States","Chave_Total retail trade (Volume) sa, IndexBrazil","Chave_Total retail trade (Volume) sa, IndexUnited States","Chave_Work started for dwellings sa, IndexUnited States"
0,Apr-2015,9967.552364,10024.214641,10367.454948,9894.336603,11647.460268
1,Apr-2016,8252.397960,9922.618280,9010.026616,10066.690822,10959.095784
2,Apr-2017,6767.212686,10369.654392,9109.530412,10538.131149,10977.926355
3,Apr-2018,4931.240235,10971.075100,9987.807719,10941.892212,13355.939510
4,Apr-2019,3997.481553,0.000000,9779.950055,11155.035183,13293.651864


In [89]:
#criando nomes para duas novas colunas
novos_nomes2 = dummies_prod_sales['Time'].str.split("-",expand=True)
dummies_prod_sales[['Mes', 'Ano']] = novos_nomes2

#Ajusta nomes dos meses
dummies_prod_sales['Mes'].replace({'Jan':'jan','Fev':'fev','Mar':'mar','Apr':'abr','May':'mai','Jun':'jun',
                                  'Jul':'jul','Aug':'ago','Sep':'set','Oct':'out','Nov':'nov','Dec':'dez'}, inplace=True)

In [90]:
#Dropa coluna Value
dummies_prod_sales.drop(dummies_prod_sales.columns[0], axis=1,inplace=True)

In [91]:
dummies_prod_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   Chave_Passenger car registrations sa, IndexUnited States                          60 non-null     float64
 1   Chave_Production of total manufactured intermediate goods sa, IndexUnited States  60 non-null     float64
 2   Chave_Total retail trade (Volume) sa, IndexBrazil                                 60 non-null     float64
 3   Chave_Total retail trade (Volume) sa, IndexUnited States                          60 non-null     float64
 4   Chave_Work started for dwellings sa, IndexUnited States                           60 non-null     float64
 5   Mes                                                                               60 non-null     object 
 6   Ano  

In [92]:
dummies_prod_sales['Ano'] = dummies_prod_sales['Ano'].astype(int)

In [95]:
cargas_atrac_agri_pib['Mes'].value_counts()

mai    224614
ago    218197
jul    217213
jun    207823
out    196648
nov    196590
mar    195009
set    194854
abr    189247
jan    186043
dez    180984
fev    171680
Name: Mes, dtype: int64

In [96]:
#Join prod_sales com as demais
cargas_atrac_agri_pib_psales = cargas_atrac_agri_pib.merge(dummies_prod_sales, how='inner', on=['Mes', 'Ano'])

#Teste para atestar a referencia do index - achamos q seja a média do ano para cada categoria - o resultado é um pouco diferente
prod_sales.loc[(prod_sales['Reference Period Code'] == '2015_100') &
              (prod_sales['Subject'] == 'Production of total industry sa, Index') &
              (prod_sales['LOCATION'] == 'USA')]['Value'].mean()

prod_sales.reset_index(drop=True, inplace=True)
prod_sales

###############################################################################################################################

############################################### Tx. Desocupação ###############################################################

In [97]:
#Não alterei nada. Tabela está boa para ser utilizada
taxa_desocupacao.head()

,Mes/Ano,Ratio
0,jan/15,6.8
1,fev/15,7.4
2,mar/15,7.9
3,abr/15,8.0
4,mai/15,8.1


In [98]:
nova_col_txocup = taxa_desocupacao['Mes/Ano'].str.split('/', expand=True)

In [99]:
taxa_desocupacao.drop(columns='Mes/Ano', inplace=True)

In [100]:
taxa_desocupacao[['Mes','Ano']] =  nova_col_txocup

In [101]:
taxa_desocupacao['Ano'] = '20' + taxa_desocupacao['Ano']
taxa_desocupacao

,Ratio,Mes,Ano
0,6.8,jan,2015
1,7.4,fev,2015
2,7.9,mar,2015
3,8.0,abr,2015
4,8.1,mai,2015
5,8.3,jun,2015
6,8.5,jul,2015
7,8.7,ago,2015
8,8.9,set,2015
9,8.9,out,2015


In [102]:
taxa_desocupacao['Ano'] = taxa_desocupacao['Ano'].astype(int)

In [103]:
taxa_desocupacao.rename(columns={'Ratio':"Taxa_desocupacao"}, inplace=True)

In [104]:
taxa_desocupacao['Taxa_desocupacao'].astype(float)

0      6.8
1      7.4
2      7.9
3      8.0
4      8.1
5      8.3
6      8.5
7      8.7
8      8.9
9      8.9
10     9.0
11     8.9
12     9.5
13    10.2
14    10.9
15    11.2
16    11.2
17    11.3
18    11.6
19    11.8
20    11.8
21    11.8
22    11.8
23    12.0
24    12.6
25    13.2
26    13.7
27    13.6
28    13.3
29    13.0
30    12.8
31    12.6
32    12.4
33    12.2
34    12.0
35    11.8
36    12.2
37    12.6
38    13.1
39    12.9
40    12.7
41    12.4
42    12.3
43    12.1
44    11.9
45    11.7
46    11.6
47    11.6
48    12.0
49    12.4
50    12.7
51    12.5
52    12.3
53    12.0
54    11.8
55    11.8
56    11.8
57    11.6
58    11.2
59    11.0
Name: Taxa_desocupacao, dtype: float64

In [105]:
cargas_atrac_agri_pib_psales_txdesoc = cargas_atrac_agri_pib_psales.merge(taxa_desocupacao, how='inner',
                                                                                       on=['Mes', 'Ano'])

###############################################################################################################################

####################################################### Infra Invest ##########################################################

Não Usaremos Infra Invest

infra_invest.columns

#Como ali em cima pegamos apenas EUA e UE, é pertinente aqui pegarmos EUA e GER (país europeu mais forte)

infra_invest = infra_invest[['Country', 'Variable', 'Measure',
       'YEAR', 'Year', 'Value']]

infra_invest = infra_invest.loc[(infra_invest['Country'].isin(['United States','Germany'])) &
                               (infra_invest['Measure']== "Euro")]

infra_invest.reset_index(drop=True, inplace=True)
infra_invest

#Ponderar que o material usado na tabela "infra" são transportados a granel.

###############################################################################################################################

################################################## Weel-being #################################################################

Não Usaremos weel being

weel_being_15.columns

weel_being_15 = weel_being_15[['TYPE_VAR', 'Indicator','Time', 'Value']]

#weel_being_15.head()

#Apenas esses indicadores pois são os únicos com 5 medições (2015-19)

weel_being_15 = weel_being_15.loc[weel_being_15['Indicator'].isin(['Social support','Feeling safe at night',
                                                                   'Households with internet access at home',
                                                                  'Employment rate'])]

weel_being_15

###############################################################################################################################

######################################################### Dolar ###############################################################

In [106]:
dolar.head()

,Data,Último,Abertura,Máxima,Mínima,Var%
0,31.12.2019,"4,0190","4,0197","4,0198","4,0194","-0,00%"
1,30.12.2019,"4,0191","4,0490","4,0490","4,0083","-0,66%"
2,27.12.2019,"4,0456","4,0566","4,0620","4,0347","-0,25%"
3,26.12.2019,"4,0557","4,0825","4,0825","4,0457","-0,69%"
4,25.12.2019,"4,0838","4,0838","4,0838","4,0838","0,01%"


In [107]:
dd_mm_aaaa = dolar['Data'].str.split('.',expand=True)

In [108]:
dolar[['Dia','Mes','Ano']] = dd_mm_aaaa
dolar.drop(columns=['Data','Último','Máxima','Mínima','Var%'], inplace=True)

In [109]:
dolar.rename(columns={'Abertura':'Valor'},inplace=True)

In [110]:
dolar['Ano'] = dolar['Ano'].astype(int) 

In [111]:
dolar.rename({'Valor':'Cotacao_Dol'},axis=1,inplace=True)

In [112]:
dolar['Mes']=dolar['Mes'].map({'01':'jan','02':'fev','03':'mar','04':'abr',
             '05':'mai','06':'jun','07':'jul','08':'ago',
             '09':'set','10':'out','11':'nov','12':'dez'})

In [113]:
dolar.head()

,Cotacao_Dol,Dia,Mes,Ano
0,"4,0197",31,dez,2019
1,"4,0490",30,dez,2019
2,"4,0566",27,dez,2019
3,"4,0825",26,dez,2019
4,"4,0838",25,dez,2019


In [114]:
cargas_atrac_agri_pib_psales_txdesoc[['Dia','Mes','Ano']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2207222 entries, 0 to 2207221
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Dia     object
 1   Mes     object
 2   Ano     int64 
dtypes: int64(1), object(2)
memory usage: 67.4+ MB


In [115]:
dolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Cotacao_Dol  1304 non-null   object
 1   Dia          1304 non-null   object
 2   Mes          1304 non-null   object
 3   Ano          1304 non-null   int32 
dtypes: int32(1), object(3)
memory usage: 35.8+ KB


In [116]:
cargas_atrac_agri_pib_psales_txdesoc_dolar = cargas_atrac_agri_pib_psales_txdesoc.merge(dolar, how='inner',
                                                                                        on=['Dia', 'Mes', 'Ano'])

In [117]:
cargas_atrac_agri_pib_psales_txdesoc_dolar.head()

,IDAtracacao,Tipo Operação da Carga,ConteinerEstado,Tipo Navegação,FlagConteinerTamanho,Sentido,TEU,Apelido Instalação Portuária,Dia,Ano,...,Chave_of which: Actual individual consumptionUS Dollar,Chave_of which: Manufacturing (ISIC rev 4)US Dollar,Chave_of which: Manufacturing (ISIC rev4)US Dollar,"Chave_Passenger car registrations sa, IndexUnited States","Chave_Production of total manufactured intermediate goods sa, IndexUnited States","Chave_Total retail trade (Volume) sa, IndexBrazil","Chave_Total retail trade (Volume) sa, IndexUnited States","Chave_Work started for dwellings sa, IndexUnited States",Taxa_desocupacao,Cotacao_Dol
0,659334,Transbordo,Cheio,Cabotagem,20,Embarcados,93,Ex-Embraport,05,2015,...,2.627439e+06,49062.426,1225540.925,9636.204272,10160.478721,10928.339798,9953.061211,9804.900988,6.8,"2,6936"
1,659334,Transbordo,Cheio,Cabotagem,40,Embarcados,130,Ex-Embraport,05,2015,...,2.627439e+06,49062.426,1225540.925,9636.204272,10160.478721,10928.339798,9953.061211,9804.900988,6.8,"2,6936"
2,659334,Movimentação de Carga,Cheio,Longo Curso,20,Embarcados,16,Ex-Embraport,05,2015,...,2.627439e+06,49062.426,1225540.925,9636.204272,10160.478721,10928.339798,9953.061211,9804.900988,6.8,"2,6936"
3,659334,Movimentação de Carga,Cheio,Longo Curso,20,Embarcados,1,Ex-Embraport,05,2015,...,2.627439e+06,49062.426,1225540.925,9636.204272,10160.478721,10928.339798,9953.061211,9804.900988,6.8,"2,6936"
4,659334,Transbordo,Cheio,Cabotagem,40,Embarcados,2,Ex-Embraport,05,2015,...,2.627439e+06,49062.426,1225540.925,9636.204272,10160.478721,10928.339798,9953.061211,9804.900988,6.8,"2,6936"


In [118]:
cargas_atrac_agri_pib_psales_txdesoc_dolar.columns.to_list()

['IDAtracacao',
 'Tipo Operação da Carga',
 'ConteinerEstado',
 'Tipo Navegação',
 'FlagConteinerTamanho',
 'Sentido',
 'TEU',
 'Apelido Instalação Portuária',
 'Dia',
 'Ano',
 'Mes',
 'Chave_var_comm_Beef and veal (cwe) - Consumption',
 'Chave_var_comm_Beef and veal (cwe) - Consumption per capita',
 'Chave_var_comm_Beef and veal (cwe) - Cow inventory',
 'Chave_var_comm_Beef and veal (cwe) - Ending stocks',
 'Chave_var_comm_Beef and veal (cwe) - Exports',
 'Chave_var_comm_Beef and veal (cwe) - Food',
 'Chave_var_comm_Beef and veal (cwe) - Imports',
 'Chave_var_comm_Beef and veal (cwe) - Producer price',
 'Chave_var_comm_Beef and veal (cwe) - Production',
 'Chave_var_comm_Beef and veal (cwe) - Trade balance',
 'Chave_var_comm_Fish - Consumption',
 'Chave_var_comm_Fish - Consumption per capita',
 'Chave_var_comm_Fish - Crush',
 'Chave_var_comm_Fish - Exports',
 'Chave_var_comm_Fish - Food',
 'Chave_var_comm_Fish - Imports',
 'Chave_var_comm_Fish - Other use',
 'Chave_var_comm_Fish - Prod

###############################################################################################################################

As linhas a seguir não contemplam nada do que foi feito acima. Abaixo apenas está linhas que restaram de outra versão.

In [ ]:
#Join os dois dfs - Cargas e Atracação

#df_joined = pd.merge(carga_cont_CAB, atrac, on='IDAtracacao', how='left')
#df_joined

In [ ]:
#Associando a sigla da Origem com o Município

#df_joined.groupby('Origem').Município.value_counts()

In [ ]:
#Associando a sigla da Origem com o Terminal

#df_joined.groupby('Origem').Terminal.value_counts()